In [0]:
import xml.etree.ElementTree as ET

In [0]:
%sh
pip install spark-xml

ERROR: Could not find a version that satisfies the requirement spark-xml (from versions: none)
ERROR: No matching distribution found for spark-xml
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-425920c6-12d2-4716-bab5-ea8af7b78da3/bin/python -m pip install --upgrade pip' command.


# Connection to blob

In [0]:
storageAccount = "datalakefabdeprojetos"
containerName = "ingrid-sollim"
mountpoint = "/mnt/ingrid-sollim/"
storageEndpoint = f"wasbs://{containerName}@{storageAccount}.blob.core.windows.net" 
storageKey = dbutils.secrets.get(scope="Kv-fab-de-proj-dev",key="kv-ingrid-sollim")
storageConn = f"fs.azure.account.key.{storageAccount}.blob.core.windows.net"
try:
    if not any(mount.mountPoint==mountpoint for mount in dbutils.fs.mounts()):
        dbutils.fs.mount(
        source = storageEndpoint,
        mount_point = mountpoint,
        extra_configs = {storageConn:storageKey}
    )
        print(f"{mountpoint} has been mounted")
    else:
        print(f"Mount point '{mountpoint}' is already mounted.")
except Exception as e:
    raise e  # Re-raise the exception if mounting fails


Mount point '/mnt/ingrid-sollim/' is already mounted.


In [0]:
path=mountpoint+"bikes"
display(dbutils.fs.ls(path))

path,name,size,modificationTime
dbfs:/mnt/ingrid-sollim/bikes/customer.csv,customer.csv,1813963,1694474860000
dbfs:/mnt/ingrid-sollim/bikes/person.csv,person.csv,13646947,1694474860000
dbfs:/mnt/ingrid-sollim/bikes/product.csv,product.csv,104823,1694474860000
dbfs:/mnt/ingrid-sollim/bikes/salesorderdetail.csv,salesorderdetail.csv,13801182,1694474860000
dbfs:/mnt/ingrid-sollim/bikes/salesorderheader.csv,salesorderheader.csv,8267704,1694474862000
dbfs:/mnt/ingrid-sollim/bikes/spoffer.csv,spoffer.csv,36680,1694474860000


In [0]:
df = spark.read.option("header",True).options(delimiter=";").csv(path+"/person.csv")
display(df.limit(10))

BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
1,EM,0,NULL,Ken,J,Sánchez,NULL,0,NULL,"""0""",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07 00:00:00.000
2,EM,0,NULL,Terri,Lee,Duffy,NULL,1,NULL,"""0""",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24 00:00:00.000
3,EM,0,NULL,Roberto,NULL,Tamburello,NULL,0,NULL,"""0""",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04 00:00:00.000
4,EM,0,NULL,Rob,NULL,Walters,NULL,0,NULL,"""0""",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28 00:00:00.000
5,EM,0,Ms.,Gail,A,Erickson,NULL,0,NULL,"""0""",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30 00:00:00.000
6,EM,0,Mr.,Jossef,H,Goldberg,NULL,0,NULL,"""0""",0DEA28FD-EFFE-482A-AFD3-B7E8F199D56F,2013-12-16 00:00:00.000
7,EM,0,NULL,Dylan,A,Miller,NULL,2,NULL,"""0""",C45E8AB8-01BE-4B76-B215-820C8368181A,2009-02-01 00:00:00.000
8,EM,0,NULL,Diane,L,Margheim,NULL,0,NULL,"""0""",A948E590-4A56-45A9-BC9A-160A1CC9D990,2008-12-22 00:00:00.000
9,EM,0,NULL,Gigi,N,Matthew,NULL,0,NULL,"""0""",5FC28C0E-6D36-4252-9846-05CAA0B1F6C5,2009-01-09 00:00:00.000
10,EM,0,NULL,Michael,NULL,Raheem,NULL,2,NULL,"""0""",CA2C740E-75B2-420C-9D4B-E3CBC6609604,2009-04-26 00:00:00.000


In [0]:
from pyspark.sql.functions import expr, col, regexp_replace,regexp_extract,when

#Check for missing data

In [0]:
#Replace NULL string for null values 
# Specify the columns to be processed
columns_to_replace_null = ["Title", "MiddleName", "Suffix", "AdditionalContactInfo"]

# Create a new DataFrame by conditionally replacing "NULL" with null values
null_replaced = df
for coluna in columns_to_replace_null:
    null_replaced = null_replaced.withColumn(coluna, when(col(coluna) == "NULL", None).otherwise(col(coluna)))

# Show the resulting DataFrame
display(null_replaced.limit(10))

BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
1,EM,0,null,Ken,J,Sánchez,null,0,null,"""0""",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07 00:00:00.000
2,EM,0,null,Terri,Lee,Duffy,null,1,null,"""0""",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24 00:00:00.000
3,EM,0,null,Roberto,null,Tamburello,null,0,null,"""0""",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04 00:00:00.000
4,EM,0,null,Rob,null,Walters,null,0,null,"""0""",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28 00:00:00.000
5,EM,0,Ms.,Gail,A,Erickson,null,0,null,"""0""",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30 00:00:00.000
6,EM,0,Mr.,Jossef,H,Goldberg,null,0,null,"""0""",0DEA28FD-EFFE-482A-AFD3-B7E8F199D56F,2013-12-16 00:00:00.000
7,EM,0,null,Dylan,A,Miller,null,2,null,"""0""",C45E8AB8-01BE-4B76-B215-820C8368181A,2009-02-01 00:00:00.000
8,EM,0,null,Diane,L,Margheim,null,0,null,"""0""",A948E590-4A56-45A9-BC9A-160A1CC9D990,2008-12-22 00:00:00.000
9,EM,0,null,Gigi,N,Matthew,null,0,null,"""0""",5FC28C0E-6D36-4252-9846-05CAA0B1F6C5,2009-01-09 00:00:00.000
10,EM,0,null,Michael,null,Raheem,null,2,null,"""0""",CA2C740E-75B2-420C-9D4B-E3CBC6609604,2009-04-26 00:00:00.000


# Clean XML column

In [0]:
display(df.select("AdditionalContactInfo").distinct())

AdditionalContactInfo
"""Additional phone numbers.750-555-8888If customer is not at work, call222-555-4444Customer uses these two additional email addresses.Customer4@sample.comCustomer4@contoso.comThis is secondary email address.Customer is expanding his sports and recreation business. Sales is negotiating large sales order with this customer. We called Joe at750-555-8888We are still working on competitive pricing. Looks promising, but we need additional marketing support to wrap this up."""
"""Additional email address.Customer6@sample.comCustomer called us. Interested in the new Touring models."""
"""These are additional phone numbers the customer provided.206-555-2222Best time to call is Monday-Wednesday mornings.209-555-2122"""
"""Additional home and mobile phone numbers:253-555-4689253-555-4878"""
"""Marketing called this customer. Customer provided this phone number for follow-up questions.305-555-8888If no answer, try this pager number.308-555-3678 and this mobile number308-555-4888"""
"""The customer is interested in the Road-450 and Mountain-500 series bicycles. Customer called us. We need to follow up. Pager 206-555-1234 is the best way to reach."""
"""These are additional phone and pager numbers for the customer.206-555-2222On weekends, contact the manager at this number.206-555-1234206-555-1244Do not page between 9:00 a.m. and 5:00 p.m.Customer provided this additional home address.P.O Box 5EdmondsWA98431USAThis is an alternative address for billing only.Joe@sample.comDo not send e-mail for urgent issues. Use telephone instead.Sales contacted this customer to explain new pricing."""
"""Sales contacted this customer for the first time at432-4444We talked about the Road bike price drop and the new spring models. Customer provided us new mobile number432-555-7809"""
"""Customer provided this telephone number for weekend access.605-555-9877secondary phone number.310-555-5432Page only if other contact attempts fail.Additional home address.990 5th AvenueRedmondWA98052USAUse this address for billing only.Customer3@sample.comDo not send e-mails for urgent issues. Use telephone instead.Customer first contacted the sales department. This customer is interested in opening a new bicycle store.Linda in sales returned the call. Customer provided another telephone number for off-hour coverage.302-555-7733We are still negotiating possible long-term contract."""
"""425-555-1112Call only after 5:00 p.m.Note that the customer has a secondary home address.123 OakSeattleWA98001USAIf correspondence to the primary address fails, try this one.Customer provided additional email address.customer1@sample.comFor urgent issues, do not send e-mail. Instead use this emergency contact phone425-555-1111.This customer is interested in purchasing high-end bicycles for his family. The customer contacted Michael in sales."""


In [0]:
df_xlm = df.withColumn("new_column", regexp_extract(df["AdditionalContactInfo"], "<act:number>(.*?)</act:number>", 1))
display(df_xlm[df_xlm["AdditionalContactInfo"]!="NULL"].limit(10))

BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate,new_column
291,SC,0,Mr.,Gustavo,NULL,Achong,NULL,2,"""425-555-1112Call only after 5:00 p.m.Note that the customer has a secondary home address.123 OakSeattleWA98001USAIf correspondence to the primary address fails, try this one.Customer provided additional email address.customer1@sample.comFor urgent issues, do not send e-mail. Instead use this emergency contact phone425-555-1111.This customer is interested in purchasing high-end bicycles for his family. The customer contacted Michael in sales.""","""-13909.13111""",D4C132D3-FCB5-4231-9DD5-888A54BEC693,2015-04-15 16:33:33.060,425-555-1112
293,SC,0,Ms.,Catherine,R.,Abel,NULL,1,"""These are additional phone and pager numbers for the customer.206-555-2222On weekends, contact the manager at this number.206-555-1234206-555-1244Do not page between 9:00 a.m. and 5:00 p.m.Customer provided this additional home address.P.O Box 5EdmondsWA98431USAThis is an alternative address for billing only.Joe@sample.comDo not send e-mail for urgent issues. Use telephone instead.Sales contacted this customer to explain new pricing.""","""-3943.7544""",D54E0552-C226-4C22-AF3B-762CA854CDD3,2015-04-15 16:33:33.077,206-555-2222
295,SC,0,Ms.,Kim,NULL,Abercrombie,NULL,0,"""Customer provided this telephone number for weekend access.605-555-9877secondary phone number.310-555-5432Page only if other contact attempts fail.Additional home address.990 5th AvenueRedmondWA98052USAUse this address for billing only.Customer3@sample.comDo not send e-mails for urgent issues. Use telephone instead.Customer first contacted the sales department. This customer is interested in opening a new bicycle store.Linda in sales returned the call. Customer provided another telephone number for off-hour coverage.302-555-7733We are still negotiating possible long-term contract.""","""-7948.6029""",F7CBDB48-0B44-470E-9F37-7060446FBFB9,2015-04-15 16:33:33.077,605-555-9877
297,SC,0,Sr.,Humberto,NULL,Acevedo,NULL,2,"""Additional phone numbers.750-555-8888If customer is not at work, call222-555-4444Customer uses these two additional email addresses.Customer4@sample.comCustomer4@contoso.comThis is secondary email address.Customer is expanding his sports and recreation business. Sales is negotiating large sales order with this customer. We called Joe at750-555-8888We are still working on competitive pricing. Looks promising, but we need additional marketing support to wrap this up.""","""-20021.0528""",5A41D336-84CF-44D7-B12B-83B64B511F7E,2015-04-15 16:33:33.090,750-555-8888
299,SC,0,Sra.,Pilar,NULL,Ackerman,NULL,0,"""Sales contacted this customer for the first time at432-4444We talked about the Road bike price drop and the new spring models. Customer provided us new mobile number432-555-7809""","""-5158.224""",DF1FB8AB-2323-4330-9AB8-54E13CE6D8F9,2015-04-15 16:33:33.090,432-4444
301,SC,0,Ms.,Frances,B.,Adams,NULL,1,"""Additional email address.Customer6@sample.comCustomer called us. Interested in the new Touring models.""","""-14477.2375""",66980082-C186-40AE-972A-74560E78C1AF,2015-04-15 16:33:33.090,
303,SC,0,Ms.,Margaret,J.,Smith,NULL,0,"""These are additional phone numbers the customer provided.206-555-2222Best time to call is Monday-Wednesday mornings.209-555-2122""","""-2421.0199""",691818E1-FC19-445F-B161-E401867368E3,2015-04-15 16:33:33.107,206-555-2222
305,SC,0,Ms.,Carla,J.,Adams,NULL,0,"""The customer is interested in the Road-450 and Mountain-500 series bicycles. Customer called us. We need to follow up. Pager 206-555-1234 is the best way to reach.""","""-22476.234""",5F183B92-1648-4948-8937-47C9C10F0583,2015-04-15 16:33:33.107,206-555-1234
307,SC,0,Mr.,Jay,NULL,Adams,NULL,1,"""Additional home and mobile phone numbers:253-555-4689253-555-4878""","""-23834.7669""",38256645-4B88-4169-B68A-5B0118F18445,2015-04-15 16:33:33.123,253-555-4689
309,SC,0,Mr.,Ronald,L.,Adina,NULL,0,"""Marketin

In [0]:
@udf
def extract_ab(xml):
    doc = ET.fromstring(xml)
    return [doc.attrib['crm:ContactRecord date'], doc.attrib['act:numbe']]

In [0]:
df_xml = df.withColumn('ab', extract_ab(df['AdditionalContactInfo']))
display(df_xml.limit(10))

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 12.0 failed 4 times, most recent failure: Lost task 0.3 in stage 12.0 (TID 31) (10.10.1.4 executor 0): org.apache.spark.api.python.PythonException: 'xml.etree.ElementTree.ParseError: syntax error: line 1, column 0', from , line 3. Full traceback below:
Traceback (most recent call last):
 File " ", line 3, in extract_ab
 File "/usr/lib/python3.9/xml/etree/ElementTree.py", line 1348, in XML
 return parser.close()
xml.etree.ElementTree.ParseError: syntax error: line 1, column 0

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:694)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:88)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:70)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:647)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:761)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:80)
	at org.apache.spark.sql.execution.collect.Collector.$anonfun$processFunc$1(Collector.scala:186)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$3(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$1(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:55)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:174)
	at org.apache.spark.scheduler.Task.$anonfun$run$4(Task.scala:137)
	at com.databricks.unity.EmptyHandle$.runWithAndClose(UCSHandle.scala:125)
	at org.apache.spark.scheduler.Task.$anonfun$run$1(Task.scala:137)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.Task.run(Task.scala:96)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$13(Executor.scala:902)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1697)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:905)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:760)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:3381)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:3313)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:3304)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:3304)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DA

In [0]:

# Convert the data into XML format
df_xml = df.withColumn('xml', expr('xml(AdditionalContactInfo)'))

# Extract information from the XML-like content
df_xml = df_xml.withColumn("parsed_xml", expr("xpath_string(xml, '/AdditionalContactInfo')"))
display(df_xml.limit(10))

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-3286741429713160> in <cell line: 3>()
      1 from pyspark.sql.functions import expr
      2 # Convert the data into XML format
----> 3 df_xml = df.withColumn('xml', expr('xml(AdditionalContactInfo)'))
      4 
      5 # Extract information from the XML-like content

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/sql/dataframe.py in withColumn(self, colName, col)
   3325         if not isinstance(col, Column):
   3326             raise TypeError("col should be Column")
-> 3327         

#Change Data types

In [0]:
null_replaced.printSchema()

root
 |-- BusinessEntityID: string (nullable = true)
 |-- PersonType: string (nullable = true)
 |-- NameStyle: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- MiddleName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- Suffix: string (nullable = true)
 |-- EmailPromotion: string (nullable = true)
 |-- AdditionalContactInfo: string (nullable = true)
 |-- Demographics: string (nullable = true)
 |-- rowguid: string (nullable = true)
 |-- ModifiedDate: string (nullable = true)



In [0]:
from  pyspark.sql.functions import col,to_timestamp
person_df = null_replaced.withColumn("BusinessEntityID",col("BusinessEntityID").cast("int"))\
.withColumn("EmailPromotion",col("EmailPromotion").cast("int"))\
.withColumn("ModifiedDate",to_timestamp(col("ModifiedDate"),"yyyy-MM-dd HH:mm:ss.SSS"))

display(person_df.limit(10))

BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
1,EM,0,null,Ken,null,Sánchez,null,0,null,"""0""",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07T00:00:00.000+0000
2,EM,0,null,Terri,null,Duffy,null,1,null,"""0""",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24T00:00:00.000+0000
3,EM,0,null,Roberto,null,Tamburello,null,0,null,"""0""",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04T00:00:00.000+0000
4,EM,0,null,Rob,null,Walters,null,0,null,"""0""",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28T00:00:00.000+0000
5,EM,0,null,Gail,null,Erickson,null,0,null,"""0""",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30T00:00:00.000+0000
6,EM,0,null,Jossef,null,Goldberg,null,0,null,"""0""",0DEA28FD-EFFE-482A-AFD3-B7E8F199D56F,2013-12-16T00:00:00.000+0000
7,EM,0,null,Dylan,null,Miller,null,2,null,"""0""",C45E8AB8-01BE-4B76-B215-820C8368181A,2009-02-01T00:00:00.000+0000
8,EM,0,null,Diane,null,Margheim,null,0,null,"""0""",A948E590-4A56-45A9-BC9A-160A1CC9D990,2008-12-22T00:00:00.000+0000
9,EM,0,null,Gigi,null,Matthew,null,0,null,"""0""",5FC28C0E-6D36-4252-9846-05CAA0B1F6C5,2009-01-09T00:00:00.000+0000
10,EM,0,null,Michael,null,Raheem,null,2,null,"""0""",CA2C740E-75B2-420C-9D4B-E3CBC6609604,2009-04-26T00:00:00.000+0000


In [0]:
person_df.printSchema()

root
 |-- BusinessEntityID: integer (nullable = true)
 |-- PersonType: string (nullable = true)
 |-- NameStyle: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- MiddleName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- Suffix: string (nullable = true)
 |-- EmailPromotion: integer (nullable = true)
 |-- AdditionalContactInfo: string (nullable = true)
 |-- Demographics: string (nullable = true)
 |-- rowguid: string (nullable = true)
 |-- ModifiedDate: timestamp (nullable = true)



In [0]:
#Rename columns
person_df = person_df.withColumnRenamed("BusinessEntityID","business_entity_id")\
    .withColumnRenamed("PersonType","person_type").withColumnRenamed("NameStyle","name_style")\
    .withColumnRenamed("Title","title").withColumnRenamed("FirstName","first_name").\
    withColumnRenamed("MiddleName","middle_name").withColumnRenamed("LastName","last_name").\
    withColumnRenamed("Suffix","suffix").withColumnRenamed("EmailPromotion","email_promotion").\
        withColumnRenamed("AdditionalContactInfo","additional_contact_info").withColumnRenamed("Demographics","demographics").withColumnRenamed("ModifiedDate","modified_date")



#Ingest the cleaned data to the database

### Connection to the database

In [0]:
# Get the secrets from the jdbc secret scope.
username = dbutils.secrets.get(scope="Kv-fab-de-proj-dev", key="kv-soll-dblogin")
password = dbutils.secrets.get(scope="Kv-fab-de-proj-dev", key="kv-soll-dbpass")

jdbcHostname = "srv-fab-projetos.database.windows.net"
jdbcPort = 1433
jdbcDatabase = "db-fab-projetos"

jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)

connectionProperties = {
  "user" : username,
  "password" : password,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

table_name = 'sollim.person'

### Insert into the database


In [0]:
person_df.write \
  .format("jdbc") \
  .option("url", jdbcUrl) \
  .option("dbtable", table_name) \
  .mode("overwrite") \
  .options(**connectionProperties) \
  .save()

In [0]:
dbutils.fs.unmount(mountpoint)

/mnt/ingrid-sollim/ has been unmounted.
Out[14]: True